# 4: Increase orographic height over the western US

Change input boundary datasets by increasing surface geopotential height by 50% in the western USA


<div class="alert alert-info">
<strong>Exercise: Increase orographic height over the western US</strong><br><br>

Create a case similar to control case but change input boundary datasets by increasing surface geopotential height by 50% in the western USA.


![Increase orographic height over the western US](../../../images/challenge/cam_topo_diff.png)

*<p style="text-align: center;"> Figure: Increase orographic height over the western US. </p>*
    
Create, configure, build and run a case called ``f2000.topo`` with the compset ``F2000climo`` at the resolution ``f09_f09_mg17`` using the same history file output as in the control.  Change the topography inpute datasets
Run for 5 days. 

</div>



<div class="alert alert-warning">  
<details>

<summary> <font face="Times New Roman" color='blue'>Click here for hints</font> </summary>
<br>

**How do I output 3 hourly instantaneous variables?**

- Use namelist variables: ``nhtfrq``, ``mfilt``, ``fincl``. 
- For more information, look at the chapter: <br>
**NAMELIST MODIFICATIONS** -> **Customize CAM output**


**Where do I change the topography dataset?**

Look at the namelist definition for the variable `bnd_topo` 

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>


**# Set environment variables** 
    
Set environment variables with the commands:
```
set CASENAME = f2000.topo
set CASEDIR = /glade/u/home/$USER/cases/$CASENAME
set RUNDIR = /glade/scratch/$USER/$CASENAME/run
set COMPSET = F2000climo
set RESOLUTION = f19_f19_mg17
```

**# Create a new case**

Create a new case with the command ``create_newcase``:
```
cd /glade/work/$USER/code/my_cesm_code/cime/scripts/
./create_newcase --case $CASEDIR --res $RESOLUTION --compset $COMPSET 
```

**# Change the job queue and account number**

If needed, change ``job queue`` and ``account number``.<br>
For instance, to run in the queue ``regular`` and the project number ``P93300642``
```  
cd $CASEDIR
./xmlchange JOB_QUEUE=regular,PROJECT=P93300642
```
This step can be redone at anytime in the process. 

**# Setup**

Invoke ``case.setup`` with the command:
```    
cd $CASEDIR
./case.setup    
```  

**# Modify orography**

Copy the topography file into your case directory and modify it over there. Here we use the nco operators but feel free to use any other tool.
```
cd $CASEDIR
cp \
/glade/p/cesmdata/cseg/inputdata/atm/cam/topo/fv_1.9x2.5_nc3000_Nsw084_Nrs016_Co120_Fi001_ZR_GRNL_031819.nc   .

ncap2 -O -s 'lat2d[lat,lon]=lat ; lon2d[lat,lon]=lon'  \
-s 'omask=(lat2d >= 30. && lat2d <= 50.) && (lon2d >= 235. && lon2d <= 260.)' \
-s 'PHIS=(PHIS*(1.+omask*0.5))' \
fv_1.9x2.5_nc3000_Nsw084_Nrs016_Co120_Fi001_ZR_GRNL_031819.nc  \
fv_1.9x2.5_nc3000_Nsw084_Nrs016_Co120_Fi001_ZR_GRNL_031819.topo50.nc
```    
 
**# Customize namelists**

Edit the file ``user_nl_cam`` and add the lines:
```
nhtfrq(2) = -3
mfilt(2) = 240
fincl2 = 'TS:I','PS:I', 'U850:I','T850:I','PRECT:I','LHFLX:I','SHFLX:I','FLNT:I','FLNS:I'
bnd_topo = '$CASEDIR/fv_1.9x2.5_nc3000_Nsw084_Nrs016_Co120_Fi001_ZR_GRNL_031819.topo50.nc'
```
You can do this with a text editor. Alternatively, you can use the echo command:
```
echo "nhtfrq(2) = -3">> user_nl_cam    
echo "mfilt(2) = 240">> user_nl_cam
echo "fincl2 = 'TS:I','PS:I', 'U850:I','T850:I','PRECT:I','LHFLX:I','SHFLX:I','FLNT:I','FLNS:I'">> user_nl_cam
echo "bnd_topo = '$CASEDIR/fv_1.9x2.5_nc3000_Nsw084_Nrs016_Co120_Fi001_ZR_GRNL_031819.topo50.nc' " >> user_nl_cam
echo "">> user_nl_cam    
```

You build the namelists with the command:
```
./preview_namelists
```
This step is optional as the script ``preview_namelists`` is automatically called by ``case.build`` and ``case.submit``. But it is nice to check that your changes made their way into:
```
$CASEDIR/CaseDocs/atm_in
```

**# Set run length**

If needed, change the ``run length``. If you want to run 5 days, you don't have to do this, as 5 days is the default. 
```   
./xmlchange STOP_N=5,STOP_OPTION=ndays
```


**# Build and submit**:
```
qcmd -- ./case.build
./case.submit
```
------------



**# Check your solution**

When the run is completed, look at the history files into the archive directory.  
    
(1) Check that your archive directory on cheyenne (The path will be different on other machines): 
```
cd /glade/scratch/$user/archive/f2000_control/atm/hist
ls 
```

As your run is only 5-day, there should be no monthly file (``h0``)

(2) Look at the contents of the ``h1`` files using ``ncdump``.

- The file should contain the instantaneous output in the file ``h1`` for the variables:
```
        float FLNS(time, lat, lon) ;
                FLNS:Sampling_Sequence = "rad_lwsw" ;
                FLNS:units = "W/m2" ;
                FLNS:long_name = "Net longwave flux at surface" ;
        float FLNT(time, lat, lon) ;
                FLNT:Sampling_Sequence = "rad_lwsw" ;
                FLNT:units = "W/m2" ;
                FLNT:long_name = "Net longwave flux at top of model" ;
        float LHFLX(time, lat, lon) ;
                LHFLX:units = "W/m2" ;
                LHFLX:long_name = "Surface latent heat flux" ;
        float PRECT(time, lat, lon) ;
                PRECT:units = "m/s" ;
                PRECT:long_name = "Total (convective and large-scale) precipitation rate (liq + ice)" ;
        float PS(time, lat, lon) ;
                PS:units = "Pa" ;
                PS:long_name = "Surface pressure" ;
        float SHFLX(time, lat, lon) ;
                SHFLX:units = "W/m2" ;
                SHFLX:long_name = "Surface sensible heat flux" ;
        float T850(time, lat, lon) ;
                T850:units = "K" ;
                T850:long_name = "Temperature at 850 mbar pressure surface" ;
        float TS(time, lat, lon) ;
                TS:units = "K" ;
                TS:long_name = "Surface temperature (radiative)" ;
        float U850(time, lat, lon) ;
                U850:units = "m/s" ;
                U850:long_name = "Zonal wind at 850 mbar pressure surface" ;
```
Note that these variables have no ``cell_methods`` attribute becasue the output is instantaneous. 

</details>
</div>

